# Qwen3 Menu Combination Evaluator (Colab)\n\nThis notebook picks one appetizer, one main dish, one drink, and one dessert, then asks Qwen3 to evaluate how good the combination is.\n\n- All prompts, inputs, and outputs are written in English.\n- You can run this directly in Google Colab.

In [ ]:
# Install dependencies (Colab)\n!pip -q install -U transformers accelerate sentencepiece

In [ ]:
import json\nimport random\nimport torch\nfrom transformers import AutoTokenizer, AutoModelForCausalLM, pipeline\n\nAPPETIZERS = [\n    "salad", "corn soup", "miso soup", "house bread", "cheese",\n    "cracker", "Scotch Egg", "mashed potato", "nachos", "pasta"\n]\n\nMAINS = [\n    "ramen", "pizza", "fried chicken", "sandwich", "T-bone steak",\n    "sushi", "taco", "grilled tofu", "fish and chips", "paella"\n]\n\nDRINKS = [\n    "coca-cola", "red wine", "white wine", "sake", "green tea",\n    "orange juice", "coke zero", "modelo beer", "coffee", "water"\n]\n\nDESSERTS = [\n    "orange", "grape", "pudding", "ice cream", "tart",\n    "cheesecake", "macaron", "dango", "muffin", "churro"\n]

In [ ]:
# Select a Qwen3 model that fits your Colab runtime (change if needed).\nMODEL_ID = "Qwen/Qwen3-4B-Instruct"\n\ntokenizer = AutoTokenizer.from_pretrained(MODEL_ID)\nmodel = AutoModelForCausalLM.from_pretrained(\n    MODEL_ID,\n    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,\n    device_map="auto"\n)\n\ngenerator = pipeline(\n    "text-generation",\n    model=model,\n    tokenizer=tokenizer\n)\n\nprint(f"Loaded model: {MODEL_ID}")

In [ ]:
def sample_menu_combo(seed=None):\n    if seed is not None:\n        random.seed(seed)\n\n    return {\n        "appetizer": random.choice(APPETIZERS),\n        "main": random.choice(MAINS),\n        "drink": random.choice(DRINKS),\n        "dessert": random.choice(DESSERTS),\n    }\n\n\ndef evaluate_combo_with_qwen3(combo, max_new_tokens=260):\n    system_prompt = (\n        "You are a food pairing expert. Evaluate one full meal combination made of appetizer, main, drink, and dessert. "\n        "Always respond in English and return valid JSON only."\n    )\n\n    user_prompt = f"""\nMeal combination to evaluate:\n- Appetizer: {combo['appetizer']}\n- Main: {combo['main']}\n- Drink: {combo['drink']}\n- Dessert: {combo['dessert']}\n\nReturn JSON with this schema exactly:\n{{\n  \"total_score\": 0-100 integer,\n  \"pairing_summary\": "short summary",\n  \"strengths\": ["...", "..."],\n  \"weaknesses\": ["...", "..."],\n  \"improvement_tip\": "one actionable tip"\n}}\n"""\n\nmessages = [\n    {"role": "system", "content": system_prompt},\n    {"role": "user", "content": user_prompt},\n]\n\nresult = generator(\n    messages,\n    max_new_tokens=max_new_tokens,\n    do_sample=True,\n    temperature=0.7,\n    top_p=0.9\n)\n\nresponse_text = result[0]["generated_text"][-1]["content"].strip()\n\n    # Attempt to parse JSON. If parsing fails, return raw text for debugging.\n    try:\n        parsed = json.loads(response_text)\n        return parsed, response_text\n    except json.JSONDecodeError:\n        return None, response_text

In [ ]:
combo = sample_menu_combo(seed=42)\nevaluation, raw_output = evaluate_combo_with_qwen3(combo)\n\nprint("Sampled combination:")\nprint(combo)\nprint("\nModel output:")\n\nif evaluation is not None:\n    print(json.dumps(evaluation, indent=2, ensure_ascii=False))\nelse:\n    print(raw_output)

In [ ]:
# Optional: evaluate multiple random combinations\nfor i in range(3):\n    combo = sample_menu_combo()\n    evaluation, raw_output = evaluate_combo_with_qwen3(combo)\n    print(f"\n=== Combo {i+1} ===")\n    print(combo)\n    if evaluation is not None:\n        print("Score:", evaluation.get("total_score"))\n        print("Summary:", evaluation.get("pairing_summary"))\n    else:\n        print("Non-JSON output:")\n        print(raw_output[:500])